# Quality Control prediction for incoming goods.
    
### **Grzegorz Kowalczyk**
#### 01/03/2024

* **1 Introduction**
* **2 Load and check data**
    * 2.1 Load data
    * 2.2 Data dictionary
    * 2.3 Data types, empty strings
    * 2.4 Outlier detection
    * 2.5 Check for null and missing values
* **4 Data preprocessing**
    * 4.1 Data cleaning
* **3 Feature analysis**
    * 3.1 Numerical values
    * 3.2 Categorical values
* **4 Filling missing Values**
    * 4.1 Age
* **5 Feature engineering**
    * 5.1 Name/Title
    * 5.2 Family Size
    * 5.3 Cabin
    * 5.4 Ticket
 
* **6 Scaling**
  
* **7 Modeling**
    * 6.1 Simple modeling
        * 6.1.1 Cross validate models
        * 6.1.2 Hyperparamater tunning for best models
        * 6.1.3 Plot learning curves
        * 6.1.4 Feature importance of the tree based classifiers
    * 6.2 Ensemble modeling
        * 6.2.1 Combining models
    * 6.3 Prediction
        * 6.3.1 Predict and Submit results

## 1. Introduction

In the context of mid-sized manufacturing enterprises, factories often engage in the production of a multitude of products, necessitating the procurement of numerous parts and components from a diverse array of international suppliers, often employing dual sourcing strategies.

This project endeavors to develop a predictive model, leveraging real-world production data, specifically focusing on aluminum LED profiles manufacturing, with the objective of accurately classifying incoming goods as either 'OK' or 'NOK' (not OK).

A robust working model, enhanced with additional refinements, holds the potential to provide valuable insights not only to quality assurance personnel but also to logistics professionals, aiding in supplier selection processes. Moreover, it can serve as a tool for research and development teams, facilitating the identification of product features during the design phase that mitigate the likelihood of faulty deliveries.


In [ ]:
import pandas as pd
# pd.set_option('display.max_rows', None)
import numpy as np
import missingno
import seaborn as sns
import warnings
from collections import Counter
warnings.filterwarnings("ignore")
# !pip install openpyxl

## 2. Load and check data

### 2.1 load data

In [ ]:
df = pd.read_excel('../Github depo/QC_encoded_data.xlsx')

In [ ]:
df.head()

In [ ]:
df.info(memory_usage='deep')

Reorder the columns

In [ ]:
new_order = ['OK_NOK','date', 'name_e', 'shape_no_e', 'drawing_no_e', 'finish', 'supplier_e', 'pcs_pck','delivered_pcs_pack', 'pack_weight_kg', 
       'pcs_lenght_mm', 'quantity_pcs', 'overal_weight_kg']

In [ ]:
df = df[new_order]

In [ ]:
df.shape

In [ ]:
df.describe()

Describe function shows only 3 columns which means only those 3 are numeric, I will address this issue later on.

## 2.2 Data dictionary

* 0.   **OK_NOK**:
      This column contains a target label   
* 1.   **date**:
      Date of delivery
* 2.   **name_e**:
      This column consist of aluminum profile names
* 3.   **shape_no_e**:
      Number that defines shape of the aluminium profile.
      Shape itself dosn't describe the final product because many suppliers can deliver the same shape in many colours and at different lenghts
* 4.   **drawing_no_e**:
      Describes a shape and supplier but dosn't contain information about colours and different lenghts
* 5.   **finish**:
      Information about the colour of the profile
* 6.   **supplier_e**:
      Name of the supplier
* 7.   **pcs_pck**:
      Number of aluminium profiles that should be delivered in one package
* 8.   **delivered_pcs_pack**:
      Number of aluminium profiles that ware delivered in one package
* 9.   **pack_weight_kg**:
      Weight of one package
* 10.  **pcs_lenght_mm**:
      Lenght of the aluminium profiles
* 11.  **quantity_pcs**:
      How many pcs where delivered 
* 12.  **overal_weight_kg**:
      The overal weight delivered profiles

## 2.3 Data types, empty strings

### Since our target is binary classification we can map OK = 1 and NOK = 0

In [ ]:
df['OK_NOK'] = df['OK_NOK'].map({'OK': 1, 'NOK': 0})

In [ ]:
df.info()

 The columns:
* 'delivered_pcs_pack',
* 'pack_weight_kg',
* 'quantity_pcs'
  
   are mix of strings, numeric and nans that I need to handle first.

* **delivered_pcs_pack'**

In [ ]:
delivered_pcs_vc = df['delivered_pcs_pack'].value_counts(dropna=False)


In [ ]:
# Change the type of whole column to str

df['delivered_pcs_pack'] = df['delivered_pcs_pack'].astype(str)

# Filter out rows containing '40 / 60'
filtered_df = df[df['delivered_pcs_pack'].str.contains('40 / 60')]


In [ ]:
#  Replace values, strip white spaces:

df['delivered_pcs_pack'] = df['delivered_pcs_pack'].replace(['20 SZT', 'BRAK', '-', '0', '40 / 60'], ['20', 'np.nan', 'np.nan', 'np.nan', '40']).str.strip()

# Change the whole column back to numeric

df['delivered_pcs_pack'] = pd.to_numeric(df['delivered_pcs_pack'], errors='coerce')


* **pack_weight_kg**

In [ ]:
# Check for strings, nan etc, then convert to nummeric.
pack_weight_kg_vc = df['pack_weight_kg'].value_counts(dropna=False)
filtered_pack_weight_kg_vc = pack_weight_kg_vc[pack_weight_kg_vc<2]


In [ ]:
df['pack_weight_kg'] =  pd.to_numeric(df['pack_weight_kg'], errors='coerce')
df['pack_weight_kg'] = df['pack_weight_kg'].round(decimals=1)
df['pack_weight_kg'].value_counts(dropna=False).index

* **quantity_pcs**

In [ ]:
quantity_pcs_vc = df['quantity_pcs'].value_counts(dropna=False)

In [ ]:
df['quantity_pcs'] = pd.to_numeric(df['quantity_pcs'], errors='coerce')

In [ ]:
df.info()

## 2.4 Outlier detection

In [ ]:
# Outlier detection 

def detect_outliers(df,n,features):
    """
    Takes a dataframe df of features and returns a list of the indices
    corresponding to the observations containing more than n outliers according
    to the Tukey method.
    """
    outlier_indices = []
    
    # iterate over features(columns)
    for col in features:
        # 1st quartile (25%)
        Q1 = np.percentile(df[col], 25)
        # 3rd quartile (75%)
        Q3 = np.percentile(df[col],75)
        # Interquartile range (IQR)
        IQR = Q3 - Q1
        
        # outlier step
        outlier_step = 1.5 * IQR
        
        # Determine a list of indices of outliers for feature col
        outlier_list_col = df[(df[col] < Q1 - outlier_step) | (df[col] > Q3 + outlier_step )].index
        
        # append the found outlier indices for col to the list of outlier indices 
        outlier_indices.extend(outlier_list_col)
        
    # select observations containing more than 2 outliers
    outlier_indices = Counter(outlier_indices)        
    multiple_outliers = list( k for k, v in outlier_indices.items() if v > n )
    
    return multiple_outliers   

# detect outliers from "pcs_pck", "pcs_lenght_mm", "overal_weight_kg", "quantity_pcs","pack_weight_kg","delivered_pcs_pack"
Outliers_to_drop = detect_outliers(df,2,["pcs_pck", "pcs_lenght_mm", "overal_weight_kg", "quantity_pcs","pack_weight_kg","delivered_pcs_pack"])

In [ ]:
Outliers_to_drop

Comment: It appears that there are no outliers in our data.

In [ ]:
df.describe()

### 2.5 Check for null and missing values

In [ ]:
df.isnull().sum().sort_values(ascending=False)

In [ ]:
missingno.matrix(df)

In [ ]:
sorted_delivered_pcs_pack = df.sort_values('delivered_pcs_pack')
missingno.matrix(sorted_delivered_pcs_pack)

### 4. Data preprocesing

### 4.1 Data cleaning

In [ ]:
df

In [ ]:
df.info(memory_usage='deep')

In [ ]:
df.date.iloc[-1]

In [ ]:
df.date.isnull().sum()

In [ ]:
df['is_str'] = df.date.apply(lambda x: True if isinstance(x, str) else False)

In [ ]:
df['is_str'].value_counts()

In [ ]:
df.insert(2, 'is_str', df.pop('is_str'))

In [ ]:
is_str_true = df.loc[df['is_str'] == True, 'date']

In [ ]:
is_str_true = is_str_true.str.replace(',','.').str.strip()

In [ ]:
is_str_true = pd.to_datetime(is_str_true, format='%d.%m.%Y', errors='coerce' )

In [ ]:
is_str_true.isnull().sum()

In [ ]:
is_str_true_idx = is_str_true[is_str_true.isnull()].index

In [ ]:
df.loc[is_str_true_idx].head(71)

In [ ]:
df.loc[is_str_true_idx, 'date'] = df.loc[is_str_true_idx, 'date'].str.replace('/','.')

In [ ]:
df.loc[72, 'date'] = '12.10.2016'
df.loc[74, 'date'] = '10.10.2016'
df.loc[233, 'date'] = '12.10.2016'
df.loc[426, 'date'] = '28.03.2016'
df.loc[894, 'date'] = '21.02.2018'
df.loc[1532, 'date'] = '12.09.2018'

In [ ]:
list_convert_to_time_indx = [72, 74, 233, 426, 894, 1532, 427, 429, 433]

In [ ]:
filtered_convert_to_time_indx = df.loc[list_convert_to_time_indx, 'date']

In [ ]:
df.date = pd.to_datetime(filtered_convert_to_time_indx, format='%d.%m.%Y', errors='coerce' )

In [ ]:
df.loc[72, 'date']

In [ ]:
df.loc[df['is_str'] == True, 'date_fmt'] = is_str_true

In [ ]:
df.insert(2, 'date_fmt', df.pop('date_fmt'))

In [ ]:
df['date_fmt'] = df['date_fmt'].dt.strftime('%Y-%m-%d %H:%M:%S')

In [ ]:
df.loc[df['is_str']==True, 'date'] = df['date_fmt']

In [ ]:
import random
from datetime import datetime, timedelta

def random_date(year):
    start_date = datetime(year, 1, 1)
    end_date = datetime(year, 12, 31)
    random_days = random.randint(0, (end_date - start_date).days)
    return start_date + timedelta(days=random_days)

In [ ]:
df.loc[is_str_true_idx, 'date'] = random_date(2016)

In [ ]:
df.loc[is_str_true_idx, 'date']

In [ ]:
g = sns.heatmap(df[["OK_NOK","pcs_pck", "pcs_lenght_mm", "overal_weight_kg", "quantity_pcs","pack_weight_kg","delivered_pcs_pack"]].corr(),annot=True, fmt = ".2f", cmap = "coolwarm")